In [27]:
#importing the libraries from pyhton
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from numpy import mean
from numpy import std
from collections import Counter

#Importing sklearn libraries
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


In [2]:
train = pd.read_csv("train.csv")
train.head(5)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1
3,F00000005,257,1,1,1,0,0.0,0,2,1
4,F00000006,342,1,0,1,0,0.0,0,2,1


In [3]:
train['Number_Weeks_Used'].fillna(train['Number_Weeks_Used'].mean(), inplace=True)

In [4]:
X = train.drop(['ID','Crop_Damage'],axis = 1)
y = train.Crop_Damage
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)

In [5]:
# This is the class split created from default (random) sample
Counter(y_train)

Counter({0: 55652, 1: 9254, 2: 1737})

In [6]:
#Importing ADASYN
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

Using TensorFlow backend.
C:\Users\kanum\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kanum\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kanum\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kanum\Anaconda3\lib\site-packages\tensorboard\comp

In [7]:
ada = SMOTE(sampling_strategy='not majority',random_state=42)
X_res, y_res = ada.fit_resample(X_train, y_train)

In [8]:
Counter(y_res)

Counter({0: 55652, 1: 55652, 2: 55652})

In [9]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate']=0.04
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=3
params['num_iterations']= 600
lgb = lgb.train(params, d_train, 100)

C:\Users\kanum\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [10]:
y_pred=lgb.predict(X_test)

In [11]:
y_pred = [np.argmax(line) for line in y_pred]

In [12]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

84.6410083277065

In [13]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92     18586
           1       0.52      0.18      0.27      3053
           2       0.00      0.00      0.00       576

    accuracy                           0.85     22215
   macro avg       0.46      0.39      0.40     22215
weighted avg       0.79      0.85      0.81     22215



In [14]:
import lightgbm as lgb
d_train_new = lgb.Dataset(X_res, label=y_res)

In [15]:
import lightgbm as lgb
d_train_new = lgb.Dataset(X_res, label=y_res)
params = {}
params['learning_rate']=0.04
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=3
params['num_iterations']= 600
lgb_s = lgb.train(params, d_train_new, 100)

In [16]:
y_pred=lgb_s.predict(X_test)

In [17]:
y_pred = [np.argmax(line) for line in y_pred]

In [18]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

80.44114337159577

In [19]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90     18586
           1       0.37      0.33      0.35      3053
           2       0.13      0.16      0.14       576

    accuracy                           0.80     22215
   macro avg       0.46      0.46      0.46     22215
weighted avg       0.80      0.80      0.80     22215



In [20]:
test = pd.read_csv("test.csv")
test.head(5)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season
0,F00000002,188,1,1,1,0,NaN,0,2
1,F00000007,410,1,1,1,0,0.0,0,2
2,F00000011,626,1,0,1,0,0.0,0,2
3,F00000013,731,1,0,1,0,0.0,0,2
4,F00000014,789,0,0,1,0,0.0,0,1


In [21]:
test['Number_Weeks_Used'].fillna(test['Number_Weeks_Used'].mean(), inplace=True)
test.isnull().sum()

ID                         0
Estimated_Insects_Count    0
Crop_Type                  0
Soil_Type                  0
Pesticide_Use_Category     0
Number_Doses_Week          0
Number_Weeks_Used          0
Number_Weeks_Quit          0
Season                     0
dtype: int64

In [22]:
new_test = test.drop(['ID'],axis = 1)

In [23]:
predictions = lgb_s.predict(new_test)

In [24]:
predictions = [np.argmax(line) for line in predictions]


In [25]:
submission = pd.read_csv("sample_submissiom.csv")
submission.head(5)

,ID,Crop_Damage
0,F00000002,1
1,F00000011,1
2,F00000014,1
3,F00000015,1
4,F00000016,1


In [26]:
submission['ID'] = test['ID']
submission['Crop_Damage'] = predictions
submission.to_csv('submission_lgb_SMOTE.csv',index=False)

In [ ]:
#CAT_Boost_Classifier
